In [ ]:
import copy
import json
import pathlib
import altair as alt
import arviz as az
import itertools
import numpy as np
import pandas as pd
import plotnine as p9
import statsmodels.api as sm
from itertools import combinations
from scipy.stats import norm

In [ ]:
def get_results(config, df, name, config_map={}):
    data_df = df.loc[df['data_name'] == name, :].copy()

    for config_name, col_name in config_map.items():
        data_df.loc[:, col_name] = data_df['data_id'].apply(lambda id: config[id][config_name])

    return data_df

def aggregate_results(df, group_cols, metric_cols, functions={'mean': ['mean'], 'median': ['median'], 'std': ['std'], 'q25': ['quantile', 0.25], 'q75': ['quantile', 0.75]}):
    summary_df = pd.DataFrame({col: [] for col in group_cols + ['variable']})
    groups_df = df.groupby(group_cols)

    for col, args in functions.items():
        tmp_df = groups_df[metric_cols].agg(*args).reset_index()
        summary_df = summary_df.merge(
            pd.melt(tmp_df, id_vars=group_cols, value_vars=metric_cols, value_name=col),
            on=group_cols + ['variable'],
            how='outer'
        )

    return summary_df

In [ ]:
def get_figure_size(width=None, height=None, ratio=None):
    match (width, height, ratio):
        case (None, None, None):
            raise ValueError('Invalid figure size parameters')
        case (width, height, None) if width is not None and height is not None:
            return (width, height)
        case (width, None, ratio) if width is not None and ratio is not None:
            return (width, width * ratio)
        case (None, height, ratio) if height is not None and ratio is not None:
            return (height / ratio, height)
        case _:
            raise ValueError('Invalid figure size parameters')

def aggregated_results_plot(df, name, x_col, y_col='median', y_min_col='q25', y_max_col='q75', model_col='model_name', var_col='variable',
    model_color_map=None, facet_cols=None, model_name_map=None, variable_map=None, facet_maps=None, scale_x_map=None, scale_x=None, labs={},
    facet_ncol=3, y_labels=True, y_limits=(0.0, None), figure_size=get_figure_size(width=6, ratio=2/3), dpi=300, errorbar_width=0.05, default_parameters=None,
    default_shape_map={'Default': 'x'}, model_shape_map=None, panel_spacing=0.025, facet_scales='fixed', guide_kwargs=None, default_y_lab='Distance metric',
    scale_y=None, dodge_width=0.2, ncol=7
):
    df = df.copy()

    if model_name_map is not None:
        df = df[df[model_col].isin(model_name_map.keys())].copy()
        df[model_col] = pd.Categorical(df[model_col].map(model_name_map), model_name_map.values())

        if model_color_map is not None:
            model_color_map = {model_name_map[k]: v for k, v in model_color_map.items() if k in model_name_map.keys()}

        if model_shape_map is not None:
            model_shape_map = {model_name_map[k]: v for k, v in model_shape_map.items() if k in model_name_map.keys()}

    if variable_map is not None:
        df = df[df[var_col].isin(variable_map.keys())].copy()
        df[var_col] = pd.Categorical(df[var_col].map(variable_map), variable_map.values())

    shape_map = model_shape_map if model_shape_map is not None else {}

    df['point_shape'] = df[model_col].copy()

    if default_parameters is not None:
        if x_col in default_parameters:
            df['is_default'] = df[x_col] == default_parameters[x_col]
            df['point_shape'] = np.where(df['is_default'], 'Default', df['point_shape'])
            shape_map = default_shape_map | shape_map
            df['point_shape'] = pd.Categorical(df['point_shape'], shape_map.keys())
        elif facet_cols is not None and np.all([col in default_parameters for col in facet_cols]):
            df['is_default'] = True

            for col in facet_cols:
                df['is_default'] = np.logical_and(df['is_default'], df[col].apply(lambda x: x == default_parameters[col]))

            df['point_shape'] = np.where(df['is_default'], 'Default', df['point_shape'])
            shape_map = default_shape_map | shape_map
            df['point_shape'] = pd.Categorical(df['point_shape'], shape_map.keys())

    if scale_x_map is not None:
        df = df[df[x_col].isin(scale_x_map.keys())].copy()
        df[x_col] = pd.Categorical(df[x_col].map(scale_x_map), scale_x_map.values())

    if facet_cols is not None:
        for facet_col in facet_cols:
            if facet_col not in facet_maps:
                continue

            facet_map = facet_maps[facet_col]
            df = df[df[facet_col].isin(facet_map.keys())].copy()
            df[facet_col] = pd.Categorical(df[facet_col].map(facet_map), facet_map.values())

    if x_col == model_col:
        df['constant'] = 'constant'
        p = p9.ggplot(df, p9.aes(x='constant', y=y_col, color=model_col, group=model_col))
    else:
        p = p9.ggplot(df, p9.aes(x=x_col, y=y_col, color=model_col, group=model_col))

    if facet_cols is not None and len(facet_cols) > 1:
        if x_col == model_col:
            df_cols = [x_col, 'constant']
        else:
            df_cols = [x_col]

        df_default_y = df.loc[df['is_default'], facet_cols + df_cols + [y_col]].copy()
        df_default = pd.merge(df.loc[:,facet_cols + df_cols], df_default_y, on=df_cols, how='left')
        p = p + p9.geom_point(p9.aes(x='constant', y=y_col), size=2, position=p9.position_dodge(width=dodge_width), shape=shape_map['Default'], color='gray', alpha=0.5, data=df_default)

    p = p + p9.geom_line()

    if y_min_col in df.columns and df[y_min_col].notna().any():
        if x_col == model_col:
            p = p + p9.geom_errorbar(p9.aes(ymin=f'{y_min_col}', ymax=f'{y_max_col}'), width=errorbar_width, position=p9.position_dodge(width=dodge_width))
        else:
            p = p + p9.geom_errorbar(p9.aes(ymin=f'{y_min_col}', ymax=f'{y_max_col}'), width=errorbar_width)

    if x_col == model_col:
        p = p + p9.geom_point(p9.aes(shape='point_shape'), size=2, position=p9.position_dodge(width=dodge_width))
    else:
        p = p + p9.geom_point(p9.aes(shape='point_shape'), size=2)

    if df[var_col].nunique() > 1:
        if facet_cols is not None and len(facet_cols) == 1:
            p = p + p9.facet_grid(f'{facet_cols[0]} ~ {var_col}', scales=facet_scales)
        else:
            p = p + p9.facet_wrap(f'~{var_col}', scales=facet_scales, ncol=facet_ncol)
    elif facet_cols is not None:
        assert len(facet_cols) < 3, 'Only up to two facet columns are supported'

        if len(facet_cols) == 1:
            p = p + p9.facet_wrap(f'~{facet_cols[0]}', scales=facet_scales, ncol=facet_ncol)
        else:
            p = p + p9.facet_grid(f'{facet_cols[0]} ~ {facet_cols[1]}')

    if scale_y == 'sqrt':
        p = p + p9.scale_y_sqrt(labels=y_labels)
    elif scale_y == 'log':
        p = p + p9.scale_y_log10(labels=y_labels)
    else:
        p = p + p9.scale_y_continuous(labels=y_labels)

    if y_limits is not None:
        p = p + p9.coord_cartesian(ylim=y_limits)

    if x_col != model_col:
        if scale_x == 'sqrt':
            p = p + p9.scale_x_sqrt()
        elif scale_x == 'log':
            p = p + p9.scale_x_log10()

    if model_color_map is not None:
        p = p + p9.scale_color_manual(breaks=list(model_color_map.keys()), values=list(model_color_map.values()))

    if shape_map:
        p = p + p9.scale_shape_manual(breaks=[k for k in shape_map.keys() if k != 'Default'], values=list(shape_map.values()))

    default_labs = {'shape': '', 'color': ''}

    if len(variable_map) > 1:
        default_labs['y'] = default_y_lab
    else:
        default_labs['y'] = next(iter(variable_map.values()))

    p = p + p9.labs(**(default_labs | labs))

    p = (p
        + p9.theme_minimal()
        + p9.theme(
            panel_spacing=panel_spacing, 
            legend_position='bottom',
            #legend_text=p9.element_text(size=8), 
            figure_size=figure_size, 
            dpi=dpi,
            panel_background=p9.element_rect(fill='white',color='white'),
            plot_background=p9.element_rect(fill='white',color='white'),
            
        )
    )

    p += p9.guides(color=p9.guide_legend(ncol=ncol), shape=p9.guide_legend(ncol=ncol))

    if x_col == model_col:
        p = p + p9.theme(axis_text_x = p9.element_blank())

    if guide_kwargs is not None:
        p = p + p9.guides(**guide_kwargs)

    if not pathlib.Path(f'../tmp/figures').exists():
        pathlib.Path(f'../tmp/figures').mkdir(parents=True, exist_ok=True)

    p.save(f'../tmp/figures/{name}.png')

    return p

In [ ]:
# Used to mark the default option on the plot with gray "X"
default_parameters = {
    'agg_bias': 0.0,
    'data_has_selection': True,
    'data_int': False,
    'error_distribution': 'normal',
    'heckman_coef_rho': 0.0,
    'heckman_rho': 0.5,
    'model_int': False,
    'model_margin': 'unit',
    'model_prior_scale': False,
    'sample_size': 1000,
    'selection_bias': 0.0,
    'selection_intercept': -1.0,
    'selection_noise_prop': 0.0,
    'selection_prop': 0.2103,
    'selection_sigma': 0.5,
    'selection_skew': 0.0,
    'outcome_noise_prop': 0.0,
    'outcome_skew': 0.0,
    'outcome_sigma': 0.5,
    'overreport_prop': 0.0,
}

In [ ]:
data_list = json.load(open('../tmp/data/data_list.json', 'r'))
data_config = {name: config for config_list in data_list.values() for name, config in config_list}

In [ ]:
model_results_df = pd.read_csv('../tmp/models/model_results.csv', index_col=0)
model_results_df['seed'] = model_results_df['data_id'].apply(lambda id: str(data_config[id].get('seed')))
print(model_results_df.shape)
print(model_results_df['data_name'].unique())
model_results_df.head()

In [ ]:
comparison_name_map = {
    '3_gg': 'default',
    '3_int_gg': 'int',
    '3_tpl_gg': 'tpl',
    '3_electoral_district_gg': 'electoral_district',
    '3_unit_gg': 'unit',
    '5_fs': 'default',
    '5_int_fs': 'int',
    '5_tpl_fs': 'tpl',
    '5_electoral_district_fs': 'electoral_district',
    '5_unit_fs': 'unit',
}

data_name_blacklist = ['est-no-selection']

metric_cols = ['kld', 'kld_1d', 'kld_2d', 'emd', 'emd_1d', 'emd_2d']

comparison_models_df = model_results_df[model_results_df['model_name'].isin(comparison_name_map.keys())].copy()
comparison_models_df['category'] = comparison_models_df['model_name'].apply(lambda x: comparison_name_map[x])
comparison_models_summary_df = comparison_models_df.groupby(['data_name', 'model_name', 'category'])[metric_cols].median()
comparison_models_summary_df = comparison_models_summary_df.reset_index().set_index('model_name')
comparison_gg_models_df = comparison_models_summary_df.loc[comparison_models_summary_df.index.str.contains('gg')].reset_index()
comparison_fs_models_df = comparison_models_summary_df.loc[comparison_models_summary_df.index.str.contains('fs')].reset_index()
comparison_models_summary_df = pd.merge(comparison_gg_models_df, comparison_fs_models_df, on=['data_name', 'category'], suffixes=('_gg', '_fs'))
comparison_models_summary_df = comparison_models_summary_df[~comparison_models_summary_df['data_name'].isin(data_name_blacklist)]

for col in metric_cols:
    comparison_models_summary_df[col] = comparison_models_summary_df[f'{col}_fs'] / comparison_models_summary_df[f'{col}_gg'] - 1
#comparison_models_summary_df[metric_cols].agg(['mean', 'median', 'std', 'min', 'max'])
#comparison_models_summary_df

In [ ]:
def get_map(all_map, names):
    return {k: v for k, v in all_map.items() if k in names}

model_name_all_map = {
    '1_bp': 'BP model',
    '2_ei': 'EI model',
    '3_gg': 'GG model',
    '4_pm': 'PM model',
    '5_fs': 'FS model'
}

model_name_map = get_map(model_name_all_map, ['3_gg', '4_pm', '5_fs'])

variable_all_map = {
    'kld': 'Full margin $D_{KL}$',
    'kld_1d': '1D margin median $D_{KL}$',
    'kld_2d': '2D margin median $D_{KL}$',
    'emd': 'Full margin $D_{EM}$',
    'emd_1d': '1D margin median $D_{EM}$',
    'emd_2d': '2D margin median $D_{EM}$'
}

model_color_map = {
    '1_bp': '#e41a1c',
    '2_ei': '#377eb8',
    '3_gg': '#4daf4a',
    '4_pm': '#984ea3',
    '5_fs': '#ff7f00'
}
model_shape_map = {
    '1_bp': 'o',
    '2_ei': '^',
    '3_gg': 's',
    '4_pm': 'D',
    '5_fs': 'v'
}

variable_map = get_map(variable_all_map, ['kld_2d'])

In [ ]:
default_df = get_results(data_config, model_results_df, 'est-default')
default_summary_df = aggregate_results(default_df, ['data_name', 'model_name'], list(variable_all_map.keys()))

aggregated_results_plot(
    default_summary_df,
    'default_parameters_linear',
    'model_name',
    model_name_map=model_name_all_map, variable_map=get_map(variable_all_map, ['kld', 'kld_1d', 'kld_2d', 'emd', 'emd_1d', 'emd_2d']),
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': '', 'y': 'Distance from ground truth'},
    errorbar_width=0.2,
    figure_size=get_figure_size(width=7, ratio=2/3),
    default_parameters=default_parameters,
    facet_scales='free_y',
    ncol=7
)

In [ ]:
aggregated_results_plot(
    default_summary_df,
    'default_parameters',
    'model_name',
    model_name_map=model_name_all_map, variable_map=get_map(variable_all_map, ['kld', 'kld_1d', 'kld_2d', 'emd', 'emd_1d', 'emd_2d']),
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': '', 'y': 'Distance from ground truth'},
    errorbar_width=0.2,
    figure_size=get_figure_size(width=7, ratio=2/3),
    default_parameters=default_parameters,
    facet_scales='free_y',
    scale_y='log',
    y_limits=(10**(-3.25), 10**(-0.5)),
    ncol=7
)

### Aggregation bias

In [ ]:
agg_bias_df = get_results(data_config, model_results_df, 'est-agg-bias', {'aggregation_bias_kwargs/coefs': 'agg_bias'})
agg_bias_df['agg_bias'] = agg_bias_df['agg_bias'].apply(lambda x: x['nationality']['Other'])
agg_bias_summary_df = aggregate_results(agg_bias_df, ['model_name', 'agg_bias'], list(variable_all_map.keys()))

aggregated_results_plot(
    agg_bias_summary_df,
    'agg_bias',
    'agg_bias',
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': 'Aggregation bias ($\\beta_{AB}$)'},
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.5))
)

### Selection bias / non-response bias

In [ ]:
no_selection_df = pd.concat([
    get_results(data_config, model_results_df, 'est-no-selection'),
    get_results(data_config, model_results_df, 'est-default'),
])
no_selection_df['data_has_selection'] = no_selection_df['data_name'] == 'est-default'
no_selection_summary_df = aggregate_results(no_selection_df, ['model_name', 'data_has_selection'], list(variable_all_map.keys()))

aggregated_results_plot(
    no_selection_summary_df,
    'no_selection',
    'data_has_selection',
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    scale_x_map={False: 'No selection effect', True: 'Selection effect'},
    labs={'x': 'Selection effect in data'},
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.5)),
    errorbar_width=0.10,
)

In [ ]:
non_response_df = get_results(data_config, model_results_df, 'est-non-response', {'heckman_coef_kwargs/heckman_coefs/selection/intercept': 'selection_intercept'})

selection_props = {}

for data_id in non_response_df['data_id'].unique():
    pop_df = pd.read_csv(f'../tmp/data/{data_id}/population.csv')
    selection_prop = pop_df['selection'].sum() / len(pop_df)
    selection_props[data_id] = selection_prop.round(3)

non_response_df['selection_prop'] = non_response_df['data_id'].apply(lambda id: selection_props[id])
non_response_summary_df = aggregate_results(non_response_df, ['model_name', 'selection_intercept'], list(variable_all_map.keys()))
selection_prop_df = aggregate_results(non_response_df, ['model_name', 'selection_intercept'], ['selection_prop'], functions={'mean': ['mean']}).drop(columns=['variable']).rename(columns={'mean': 'selection_prop'})
non_response_summary_df = pd.merge(non_response_summary_df, selection_prop_df, on=['model_name', 'selection_intercept'], how='left')

aggregated_results_plot(
    non_response_summary_df,
    'non_response',
    'selection_prop',
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': 'Proportion of available population for survey sampling'},
    errorbar_width=0.025,
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.0)),
)

In [ ]:
margin_name_map = {
    "2_margin_tpl_ei": "2_ei",
    "3_margin_tpl_gg": "3_gg",
    "4_margin_tpl_pm": "4_pm",
    "5_margin_tpl_fs": "5_fs",
    "2_margin_electoral_district_ei": "2_ei",
    "3_margin_electoral_district_gg": "3_gg",
    "4_margin_electoral_district_pm": "4_pm",
    "5_margin_electoral_district_fs": "5_fs",
    "2_margin_unit_ei": "2_ei",
    "3_margin_unit_gg": "3_gg",
    "4_margin_unit_pm": "4_pm",
    "5_margin_unit_fs": "5_fs"
}

margin_map = {
    "2_margin_tpl_ei": "topline",
    "3_margin_tpl_gg": "topline",
    "4_margin_tpl_pm": "topline",
    "5_margin_tpl_fs": "topline",
    "2_margin_electoral_district_ei": "electoral_district",
    "3_margin_electoral_district_gg": "electoral_district",
    "4_margin_electoral_district_pm": "electoral_district",
    "5_margin_electoral_district_fs": "electoral_district",
    "2_margin_unit_ei": "unit",
    "3_margin_unit_gg": "unit",
    "4_margin_unit_pm": "unit",
    "5_margin_unit_fs": "unit",
}


In [ ]:
margin_df = get_results(data_config, model_results_df, 'est-electoral-district')
margin_df = margin_df[margin_df['model_name'].isin(margin_name_map.keys())]
margin_df['model_margin'] = margin_df['model_name'].apply(lambda x: margin_map[x])
margin_df['model_name'] = margin_df['model_name'].apply(lambda x: margin_name_map[x] if x in margin_name_map else x)

margin_summary_df = aggregate_results(margin_df, ['model_name', 'model_margin'], list(variable_all_map.keys()))

aggregated_results_plot(
    margin_summary_df,
    'margin',
    'model_margin',
    model_name_map=get_map(model_name_all_map, ['2_ei', '3_gg', '4_pm', '5_fs', '6_afs', '7_oppm', '8_opafs']), variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': 'Turnout margins used by model'},
    scale_x_map={'topline': 'Topline', 'electoral_district': 'Electoral district', 'unit': 'Municipality'},
    errorbar_width=0.05,
    figure_size=get_figure_size(width=5, ratio=2/3),
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.25), 10**(-1.0)),
)

### Measurement bias / overreporting bias

In [ ]:
overreport_df = get_results(data_config, model_results_df, 'est-overreport-const', {'selection_kwargs/bias/offset': 'selection_bias', 'selection_kwargs/bias/type': 'selection_bias_type'})
survey_cols = ['age_group', 'education', 'gender', 'nationality', 'electoral_district', 'unit', 'selection_error', 'outcome_error']

overreport_prop = {}
for data_id in overreport_df['data_id'].unique():
    pop_df = pd.read_csv(f'../tmp/data/{data_id}/population.csv')
    survey_df = pd.read_csv(f'../tmp/data/{data_id}/estonia_selection.csv')
    combined_df = pd.merge(survey_df, pop_df, on=survey_cols, how='left', suffixes=('', '_pop'))
    assert combined_df['outcome_pop'].isna().sum() == 0
    overreport_diff = combined_df['outcome'].sum() - combined_df['outcome_pop'].sum()
    overreport_prop[data_id] = overreport_diff / len(combined_df)

overreport_df['overreport_prop'] = overreport_df['data_id'].apply(lambda id: overreport_prop[id])

In [ ]:
overreport_summary_df = aggregate_results(overreport_df, ['model_name', 'selection_bias_type', 'selection_bias'], list(variable_all_map.keys()))
overreport_prop_df = aggregate_results(overreport_df, ['model_name', 'selection_bias_type', 'selection_bias'], ['overreport_prop'], functions={'mean': ['mean']}).drop(columns=['variable']).rename(columns={'mean': 'overreport_prop'})
overreport_summary_df = pd.merge(overreport_summary_df, overreport_prop_df, on=['model_name', 'selection_bias_type', 'selection_bias'], how='left')

aggregated_results_plot(
    overreport_summary_df[overreport_summary_df['selection_bias_type'] == 'constant'],
    'overreport_bias',
    'overreport_prop',
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': 'Proportion of mis-reporting survey responders'},
    figure_size=get_figure_size(width=5, ratio=2/3),
    default_parameters=default_parameters,
    facet_scales='free_y',
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.0)),
    errorbar_width=0.01,
)

### Collinearity

In [ ]:
hcoef_cor_df = get_results(data_config, model_results_df, 'est-hcoef-cor', {'heckman_coef_kwargs/rho': 'heckman_coef_rho'})
hcoef_cor_summary_df = aggregate_results(hcoef_cor_df, ['model_name', 'heckman_coef_rho'], list(variable_all_map.keys()))

aggregated_results_plot(
    hcoef_cor_summary_df,
    'hcoef_cor',
    'heckman_coef_rho',
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': 'Selection and outcome process correlation ($\\rho_H$)'},
    errorbar_width=0.025,
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.5)),
)

### Non-normal errors

In [ ]:
non_normal_error_df = get_results(data_config, model_results_df, 'est-non-normal-error', {'heckman_coef_kwargs/heckman_coefs/error/distribution': 'error_distribution', 'heckman_coef_kwargs/heckman_coefs/error/dof': 'error_dof', 'heckman_coef_kwargs/heckman_coefs/error/skew': 'error_skew'})
non_normal_error_df['selection_skew'] = non_normal_error_df['error_skew'].apply(lambda x: x[0])
non_normal_error_df['outcome_skew'] = non_normal_error_df['error_skew'].apply(lambda x: x[1])
default_df = get_results(data_config, model_results_df, 'est-default')
default_df['error_distribution'] = 'normal'

non_normal_error_tail_df = pd.concat([
    non_normal_error_df.loc[np.logical_and(non_normal_error_df['selection_skew'] == 0.0, non_normal_error_df['outcome_skew'] == 0.0)],
    default_df,
])

In [ ]:
non_normal_error_tail_summary_df = aggregate_results(non_normal_error_tail_df, ['data_name', 'model_name', 'error_distribution'], list(variable_all_map.keys()))

aggregated_results_plot(
    non_normal_error_tail_summary_df,
    'non_normal_error_tail',
    'error_distribution',
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    scale_x_map={'normal': 'Normal', 'skewed_t': 'Student\'s t'},
    labs={'x': 'Heckman error distribution'},
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.75)),
)

In [ ]:
non_normal_error_summary_df = aggregate_results(non_normal_error_df, ['model_name', 'selection_skew', 'outcome_skew'], list(variable_all_map.keys()))

aggregated_results_plot(
    non_normal_error_summary_df,
    'non_normal_error',
    'model_name',
    facet_cols=['selection_skew', 'outcome_skew'],
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    facet_maps={'selection_skew': {x: 'Sel: $\\gamma^s$ = %s' % x for x in non_normal_error_summary_df['selection_skew'].unique()}, 'outcome_skew': {x: 'Out: $\\gamma^o$ = %s' % x for x in non_normal_error_summary_df['outcome_skew'].unique()}},
    labs={'x': ''},
    errorbar_width=0.25,
    figure_size=get_figure_size(width=7, ratio=1/2),
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.5)),
    dodge_width=0.4,
)

### Error correlation

In [ ]:
heck_cor_df = get_results(data_config, model_results_df, 'est-heck-cor', {'heckman_coef_kwargs/heckman_coefs/error/rho': 'heckman_rho'})
heck_cor_summary_df = aggregate_results(heck_cor_df, ['model_name', 'heckman_rho'], list(variable_all_map.keys()))

aggregated_results_plot(
    heck_cor_summary_df,
    'heck_cor',
    'heckman_rho',
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': 'Heckman error correlation'},
    errorbar_width=0.025,
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.0)),
)

### Selection and outcome effect size

In [ ]:
hcoef_sigma_df = get_results(data_config, model_results_df, 'est-hcoef-sigma', {'heckman_coef_kwargs/default_sigma/selection': 'selection_sigma', 'heckman_coef_kwargs/default_sigma/outcome': 'outcome_sigma'})
hcoef_sigma_summary_df = aggregate_results(hcoef_sigma_df, ['model_name', 'selection_sigma', 'outcome_sigma'], list(variable_all_map.keys()))

aggregated_results_plot(
    hcoef_sigma_summary_df,
    'hcoef_sigma_outcome',
    'model_name',
    model_name_map=model_name_all_map, variable_map=get_map(variable_all_map, ['kld_2d']),
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    facet_cols=['selection_sigma', 'outcome_sigma'],
    facet_maps={'selection_sigma': {x: 'Sel: $\\sigma_{H,S}$ = %s' % x for x in hcoef_sigma_df['selection_sigma'].unique()}, 'outcome_sigma': {x: 'Out: $\\sigma_{H,O}$ = %s' % x for x in hcoef_sigma_df['outcome_sigma'].unique()}},
    labs={'x': ''},
    errorbar_width=0.3,
    figure_size=get_figure_size(width=7, ratio=2/3),
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.5), 10**(-0.5)),
    dodge_width=0.5,
)

### Selection and outcome effect interactions

In [ ]:
int_data_df = get_results(data_config, model_results_df, 'est-int')

int_model_names = [
    '1_bp',
    '2_ei',
    '3_gg',
    '4_pm',
    '5_fs',
    '1_int_bp',
    '2_int_ei',
    '3_int_gg',
    '4_int_pm',
    '5_int_fs',
]

int_name_map = {
    "1_int_bp": "1_bp",
    "2_int_ei": "2_ei",
    "3_int_gg": "3_gg",
    "4_int_pm": "4_pm",
    "5_int_fs": "5_fs",
}

int_data_df['model_int'] = int_data_df['model_name'].apply(lambda x: x in int_name_map)
int_data_df['model_name'] = int_data_df['model_name'].apply(lambda x: int_name_map[x] if x in int_name_map else x)
int_data_df['data_int'] = True

non_int_data_df = get_results(data_config, model_results_df, 'est-default')
non_int_data_df = non_int_data_df[non_int_data_df['model_name'].isin(int_model_names)]
non_int_data_df['model_int'] = non_int_data_df['model_name'].apply(lambda x: x in int_name_map)
non_int_data_df['model_name'] = non_int_data_df['model_name'].apply(lambda x: int_name_map[x] if x in int_name_map else x)
non_int_data_df['data_int'] = False

int_df = pd.concat([int_data_df, non_int_data_df])
int_df['interaction_case'] = int_df.apply(lambda row: f'{"with_data_int" if row["data_int"] else "without_data_int"}-{"with_model_int" if row["model_int"] else "without_model_int"}', axis=1)

In [ ]:
int_summary_df = aggregate_results(int_df, ['model_name', 'model_int', 'data_int'], list(variable_all_map.keys()))

aggregated_results_plot(
    int_summary_df,
    'int',
    'model_name',
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': 'Two-way interactions between inputs'},
    facet_cols=['model_int', 'data_int'],
    facet_maps={'model_int': {x: 'Model interactions' if x else 'No model interactions' for x in int_summary_df['model_int'].unique()}, 
    'data_int': {x: 'Data interactions' if x else 'No data interactions' for x in int_summary_df['data_int'].unique()}},
    errorbar_width=0.15,
    figure_size=get_figure_size(width=5, ratio=2/3),
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.75)),
    dodge_width=0.4,
)

### Random noise

In [ ]:
noise_df = get_results(data_config, model_results_df, 'est-noise', {'heckman_coef_kwargs/heckman_coefs/selection_noise_prop': 'selection_noise_prop', 'heckman_coef_kwargs/heckman_coefs/outcome_noise_prop': 'outcome_noise_prop'})
noise_summary_df = aggregate_results(noise_df, ['model_name', 'selection_noise_prop', 'outcome_noise_prop'], list(variable_all_map.keys()))

aggregated_results_plot(
    noise_summary_df,
    'noise_outcome',
    'model_name',
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': ''},
    facet_cols=['selection_noise_prop', 'outcome_noise_prop'],
    facet_maps={'selection_noise_prop': {x: f'Sel. noise: {100*x:.0f}%' for x in noise_summary_df['selection_noise_prop'].unique()}, 'outcome_noise_prop': {x: f'Out. noise: {100*x:.0f}%' for x in noise_summary_df['outcome_noise_prop'].unique()}},
    errorbar_width=0.15,
    figure_size=get_figure_size(width=7, ratio=1/2),
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.5)),
    dodge_width=0.4,
)

### Sample size

In [ ]:
sample_size_df = get_results(data_config, model_results_df, 'est-sample-size', {'selection_kwargs/sample_size': 'sample_size'})
sample_size_summary_df = aggregate_results(sample_size_df, ['model_name', 'sample_size'], list(variable_all_map.keys()))

aggregated_results_plot(
    sample_size_summary_df,
    'sample_size',
    'sample_size',
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': 'Sample size'},
    errorbar_width=50,
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.75)),
)

### Prior scale

In [ ]:
scale_model_names = [
    '1_bp',
    '2_ei',
    '3_gg',
    '4_pm',
    '5_fs',
    '1_scale_bp',
    '2_scale_ei',
    '3_scale_gg',
    '4_scale_pm',
    '5_scale_fs'
]

scale_name_map = {
    '1_scale_bp': '1_bp',
    '2_scale_ei': '2_ei',
    '3_scale_gg': '3_gg',
    '4_scale_pm': '4_pm',
    '5_scale_fs': '5_fs',
}

prior_df = get_results(data_config, model_results_df, 'est-default')
prior_df = prior_df[prior_df['model_name'].isin(scale_model_names)]
prior_df['model_prior_scale'] = prior_df['model_name'].apply(lambda x: x in scale_name_map)
prior_df['model_name'] = prior_df['model_name'].apply(lambda x: scale_name_map[x] if x in scale_name_map else x)

In [ ]:
prior_summary_df = aggregate_results(prior_df, ['model_name', 'model_prior_scale'], list(variable_all_map.keys()))

aggregated_results_plot(
    prior_summary_df,
    'prior_scale',
    'model_prior_scale',
    model_name_map=model_name_all_map, variable_map=variable_map,
    model_color_map=model_color_map, model_shape_map=model_shape_map,
    labs={'x': 'Prior type'},
    scale_x_map={False: 'Weakly informative priors', True: 'Strongly informative priors'},
    errorbar_width=0.05,
    figure_size=get_figure_size(width=5, ratio=2/3),
    default_parameters=default_parameters,
    scale_y='log',
    y_limits=(10**(-3.0), 10**(-0.75)),
)

### Divergences and R-hats

In [ ]:
import seaborn as sns

pd.set_option('display.float_format','{:,.2f}'.format)

def config_without_seed(config, data_id):
    conf = dict(config[data_id])
    del conf['seed']
    return ','.join([f'{key}={value}' for key, value in conf.items()])

def get_description(df, cols):
    data_df = df.copy()
    data_df['description'] = data_df.apply(lambda row: row['data_name'] + '[' + ', '.join([f'{col}={row[col]}' for col in cols]) + ']', axis=1)
    return data_df

results_df = pd.concat([
    get_description(default_df, []),
    get_description(agg_bias_df, ['agg_bias']),
    get_description(no_selection_df, ['data_has_selection']),
    get_description(non_response_df, ['selection_intercept']),
    get_description(margin_df, ['model_margin']),
    get_description(overreport_df, ['selection_bias_type', 'selection_bias']),
    get_description(hcoef_cor_df, ['heckman_coef_rho']),
    get_description(non_normal_error_tail_df, ['error_distribution']),
    get_description(non_normal_error_df, ['selection_skew', 'outcome_skew']),
    get_description(heck_cor_df, ['heckman_rho']),
    get_description(hcoef_sigma_df, ['selection_sigma', 'outcome_sigma']),
    get_description(int_df, ['model_int', 'data_int']),
    get_description(noise_df, ['selection_noise_prop', 'outcome_noise_prop']),
    get_description(sample_size_df, ['sample_size']),
    get_description(prior_df, ['model_prior_scale']),
])

#model_results_df.groupby(['data_name', 'description', 'model_name']).divergences.agg(['mean', 'std', 'median', 'min', 'max'])
#model_results_df.groupb

#pd.pivot_table(model_results_df, values='divergences', index=['data_name', 'description', 'model_name'], aggfunc='median')

def model_name_remap(model_name):
    if model_name.endswith('_bp'):
        return '01_bp'
    elif model_name.endswith('_ei'):
        return '02_ei'
    elif model_name.endswith('_gg'):
        return '03_gg'
    elif model_name.endswith('_pm'):
        return '04_pm'
    elif model_name.endswith('_fs'):
        return '05_fs'


results_df['model_name_s'] = results_df['model_name'].apply(model_name_remap)

#results_df.loc[results_df['model_name_s'] == '2_ei', 'divergences'] = results_df.loc[results_df['model_name_s'] == '2_ei', 'divergences'] / 2.0
#results_df.loc[results_df['model_name_s'] == '10_pmdei', 'divergences'] = results_df.loc[results_df['model_name_s'] == '10_pmdei', 'divergences'] / 2.0

(
    p9.ggplot(results_df.groupby(['description', 'model_name_s']).divergences.median().reset_index(), p9.aes(y='description', x='model_name_s', fill='divergences'))
    + p9.geom_tile(width=0.95, height=0.95)
    + p9.geom_text(p9.aes(label='divergences'), color='black')
    + p9.scale_fill_gradientn(colors=sns.color_palette("RdYlGn_r", 100))
    + p9.theme(
        figure_size=get_figure_size(width=12, ratio=1.0),
    )
)

In [ ]:
(
    p9.ggplot(results_df.groupby(['description', 'model_name_s']).mean_rhat.median().round(3).reset_index(), p9.aes(y='description', x='model_name_s', fill='mean_rhat'))
    + p9.geom_tile(width=0.95, height=0.95)
    + p9.geom_text(p9.aes(label='mean_rhat'), color='black')
    + p9.scale_fill_gradientn(colors=sns.color_palette("RdYlGn_r", 100))
    + p9.theme(
        figure_size=get_figure_size(width=14, ratio=1.0),
    )
)